<a href="https://colab.research.google.com/github/marrin27/2024_02_29_SIN2324/blob/main/TL%20Sessions%20de%20laboratori/S2%20Perceptr%C3%B3/openml_credit-g.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Perceptró aplicat a olivetti


**Lectura del corpus:** $\;$ comprovem també que les matrius de dades i etiquetes tenes les files i columnes que toca

**Partició del corpus:** $\;$ Creem un split de digits amb un $20\%$ de dades per a test i la resta per a entrenament (training), barallant prèviament les dades d'acord amb una llavor donada per a la generació de nombres aleatoris. Ací, com en tot codi que incloga aleatorietat (que requerisca generar nombres aleatoris), convé fixar la dita llavor per a poder reproduir experiments amb exactitud.

In [5]:
import numpy as np; from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split

digits = fetch_olivetti_faces(); X = digits.data.astype(np.float16); X/=X.max() # esto ultimo normaliza los valores para estar entre 0 y 1
y = digits.target.astype(np.uint).reshape(-1, 1); # convierte las etiquetas de clases en enteros sin signo, y reorganiza el array para que sea de 64x1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
print(X_train.shape, X_test.shape)
# print(X.shape, y.shape, "\n", np.hstack([X, y])[:5, :]) imprime las 5 primeras filas, con los valores de sus atributos + su etiqueta de clase

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data
(320, 4096) (80, 4096)


Como funciona reshape con un array sencillo


In [ ]:
import numpy as np; A=np.array([[1,2],[3,4]]).reshape(4,1); A

array([[1],
       [2],
       [3],
       [4]])

**Implementació de Perceptró:** $\;$ torna pesos en notació homogènia, $\mathbf{W}\in\mathbb{R}^{(1+D)\times C};\;$ també el nombre d'errors i iteracions executades

In [6]:
def perceptro(X, y, b=0.1, a=1.0, K=200):
    N, D = X.shape; Y = np.unique(y); C = Y.size; W = np.zeros((1+D, C))
    for k in range(1, K+1):
        E = 0
        for n in range(N):
            xn = np.array([1, *X[n, :]])
            cn = np.squeeze(np.where(Y==y[n]))
            gn = W[:,cn].T @ xn; err = False
            for c in np.arange(C):
                if c != cn and W[:,c].T @ xn + b >= gn:
                    W[:, c] = W[:, c] - a*xn; err = True
            if err:
                W[:, cn] = W[:, cn] + a*xn; E = E + 1
        if E == 0:
            break;
    return W, E, k

**Aprenentatge d'un classificador (lineal) amb Perceptró:** $\;$ Perceptró minimitza el nombre d'errors d'entrenament (amb marge)
$$\mathbf{W}^*=\operatorname*{argmin}_{\mathbf{W}=(\boldsymbol{w}_1,\dotsc,\boldsymbol{w}_C)}\sum_n\;\mathbb{I}\biggl(\max_{c\neq y_n}\;\boldsymbol{w}_c^t\boldsymbol{x}_n+b \;>\; \boldsymbol{w}_{y_n}^t\boldsymbol{x}_n\biggr)$$

In [7]:
W, E, k = perceptro(X_train, y_train)
print("Nombre d'iteracions executades: ", k)
print("Nombre d'errors d'entrenament: ", E)
print("Vectors de pesos de les classes (en columnes i amb notació homogènia):\n", W);

Nombre d'iteracions executades:  42
Nombre d'errors d'entrenament:  0
Vectors de pesos de les classes (en columnes i amb notació homogènia):
 [[-587.         -581.         -581.         ... -590.
  -557.         -585.        ]
 [-247.08929443 -233.21960449 -244.16125488 ... -266.64141846
  -244.67816162 -239.23931885]
 [-265.13061523 -251.18511963 -267.29406738 ... -281.07702637
  -270.37103271 -256.94555664]
 ...
 [-211.28652954 -203.89273071 -199.72198486 ... -196.6746521
  -201.48944092 -207.01318359]
 [-205.98419189 -202.98080444 -198.18414307 ... -194.52960205
  -195.22445679 -199.60604858]
 [-206.72164917 -203.47958374 -198.56820679 ... -189.42913818
  -192.849823   -194.80505371]]


**Càlcul de la taxa d'error en test:**

In [8]:
X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
print(f"Taxa d'error en test: {err_test:.1%}")

Taxa d'error en test: 7.5%


In [ ]:
np.argmax([[2,3,1],[4,7,1]])

4

**Ajust del marge:** $\;$ experiment per a aprendre un valor de $b$

In [9]:
for b in (.0, .01, .1, 10, 100):
    W, E, k = perceptro(X_train, y_train, b=b, K=1000)
    X_testh = np.hstack([np.ones((len(X_test), 1)), X_test])
    y_test_pred  = np.argmax(X_testh @ W, axis=1).reshape(-1, 1)
    err_test = np.count_nonzero(y_test_pred != y_test) / len(X_test)
    print(f"b: {b} E: {E} k: {k} err_test: {err_test:.1%}")

b: 0.0 E: 0 k: 48 err_test: 18.8%
b: 0.01 E: 0 k: 58 err_test: 11.2%
b: 0.1 E: 0 k: 42 err_test: 7.5%
b: 10 E: 0 k: 45 err_test: 15.0%
b: 100 E: 0 k: 72 err_test: 8.8%


**Interpretació de resultats:** $\;$ les dades d'entrenament no semblen linealment separables; no està clar que un marge major que zero puga millorar resultats, sobretot perquè sols tenim $30$ mostres de test; amb marge nul ja hem vist que s'obté un error (en test) del $16.7\%$